In [115]:
%%javascript
OutputArea.prototype._should_scroll = function(){return false}

<IPython.core.display.Javascript object>

# <i class="fa fa-folder-open" style="font-size: 2em;"></i> Open Data and Sources
***

<img src="https://media.springernature.com/original/springer-static/image/art%3A10.1007%2Fs10764-016-9925-x/MediaObjects/10764_2016_9925_Fig1_HTML.gif" width="200" height="200" align="right" />


## <i class="fa fa-database" style="font-size: 1.5em;"></i> &nbsp;  You can download the data used in this study from [here](http://neuropoly.pub/pigHeartsData)

-  <font color="Teal">Raw DICOM data (.dcm)</font>
-  <font color="Teal">Data in native Octave/MATLAB (.mat) format</font> 

<img src="https://cdn4.iconfinder.com/data/icons/iconsimple-logotypes/512/github-512.png" width="110" height="110" align="right"/>
## <i class="fa fa-cloud-download" style="font-size: 1.5em;"></i> &nbsp; You can let  MATLAB download the data and repreat the statistical analysis. 

### Access source code on GitHub: http://neuropoly.pub/pigHeartsSrc

Simply run <font color="Teal" size =4>**pigExvivo.m**</font>  (under MATLABSource folder) main script. This will:
- Dowload the data hosted on OSF (MATLAB version > R2014b) 
- Parse them into MATLAB struct for easier use  
- Perform statistical analysis and export MATLAB generated (static) figures 
- Export .mat files, which are provided as input to this notebook. 






<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/3/37/Plotly-logo-01-square.png/220px-Plotly-logo-01-square.png" width="140" height="140" align="right" />

# <i class="fa fa-compass" style="font-size: 1.5em;"></i><left> &nbsp; Explore data with interactive figures. </left> 
***
<font color="Teal" size="4"><left>You can interact with figures right away.</left></font>
<br><br>
<font color="Black" size="4"><left>Moreover, this notebook contains 6 (hidden) Python code cells that enables you to regenerate figures.</left></font>

<br>
<i class="fa fa-exclamation-circle" style="font-size: 2em;"></i> &nbsp; **Regenerating figures**
- If want to regenerate figures, or to modify them, you need to set your <font color="Teal">**Plotly credentials**</font> in the cell below. 

- If you don't have one, **you can create a free plotly account!** Please find details regarding setting credentials [here](https://plot.ly/python/getting-started/).

In [105]:
import plotly
#plotly.tools.set_credentials_file(username='your_user_name', api_key='your_api_key')
plotly.tools.set_credentials_file(username='your_user_name', api_key='your_api_key')

# After setting your credentials, you can display hidden codes and execute cells one by one. 
# Please run the cells in succession. 

## <font color="Teal"> Interact with boxplots to see how quantitative metrics per method vary over weeks  </font>
***
- [Robust correlation toolbox](https://sourceforge.net/p/robustcorrtool/wiki/Home/) uses interquartile range (IQR) based bivariate outlier removal. An intuitive explanation for this outlier labeling can be found [here](https://help.plot.ly/what-is-a-box-plot/).
- Data points detected as outliers (you can repeat this analysis on MATLAB, please see above) are discarded from the boxplot visualizations below for **MTR** and **T2** to visualize their effect on respective univariate distributions.
<br>

<i class="fa fa-info-circle" style="font-size: 1.5em;"></i> &nbsp;  You can use buttons on the left to navigate through the data.

For example, see how diferent cardiac T1 mapping methods (_MOLLI, SHMOLLI and SASHA_) consistently over- or underestimate T1 values at all weeks with respect to the goldstandard Inversion Recovery (_IR_) technique.

***


In [116]:
from IPython.display import IFrame
IFrame('//plot.ly/~agkar/136.embed', width=1000, height=600)

<center><i class="fa fa-sort-down" style="font-size: 1.5em;"></i></center>

In [87]:
import numpy as np
import scipy.io
from scipy.io import loadmat
import plotly.plotly as py
import plotly.graph_objs as go
import ipywidgets as widgets
from IPython.display import display, clear_output
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual, ToggleButtons, Layout, Button, Box, VBox, Label, HBox
from collections import namedtuple
import warnings
warnings.filterwarnings('ignore')

# ----------------------------------------------------------------------------------------- NOTE
# You can regenerate this figure if your plotly credentials are set as describde above. 
# This is the first cell, so just uncomment the last line (remove the #) and hit <Ctrl + Enter>
# ----------------------------------------------------------------------------------------- 

# Load data in native MATLAB/Octave format (.mat) 
# You can re-generate these files by running MATLABSource/pigExVivo.m 

# Part of this notebook makes use of pigMyocardium.mat to provide examples of field access in python environment.
# Note that the same data is available (parsed differently) in exvivoCorrelation.mat

mat = loadmat('phantomData.mat', struct_as_record=False)
phantomData = mat['phantomData'];

mat = loadmat('pigMyocardium.mat', struct_as_record=False)
pigMyocardium = mat['pigMyocardium'];


def getHearts(pigMyocardium,method,week,exclude):
    a = [];
    hvr = [];
    for hr_iter in range(0,6):
        if ((((week == 2) and (hr_iter==2 or hr_iter==3)) or ((week == 5 or week==6) and (hr_iter==0))) and (exclude==1))== False:
            exec('a.append(np.mean(np.array(pigMyocardium[0,hr_iter].Week' + str(week) + '[0,0].' + method + 'vec)))')
            hvr.append('heart: ' + str(hr_iter+1))           
             
    return  (np.array(a), hvr)

def getBox(pigMyocardium,method,week,color,exclude):
    week = go.Box(
        y = getHearts(pigMyocardium,method,week,exclude)[0],
        name = "Week " + str(week),
        jitter = 0.3,
        pointpos = -1.8,
        boxpoints = 'all',
        marker = dict(
            color = color,
        ),
        line = dict(
            color = color)
        
    )
    return week
wk1color = 'rgb(27,231,255)'
wk2color = 'rgb(255,65,54)'
wk3color = 'rgb(255,133,27)'
wk4color =  'rgb(61,153,112)'
wk5color = 'rgb(224,108,159)'
wk6color = 'rgb(110,235,131)'
def getAllWeeks(pigMyocardium,method,exclude):
    wk1 = getBox(pigMyocardium,method,1,wk1color,exclude)
    wk2 = getBox(pigMyocardium,method,2,wk2color,exclude)
    wk3 = getBox(pigMyocardium,method,3,wk3color,exclude)
    wk4 = getBox(pigMyocardium,method,4,wk4color,exclude)
    wk5 = getBox(pigMyocardium,method,5,wk5color,exclude)
    wk6 = getBox(pigMyocardium,method,6,wk6color,exclude)


    return [wk1, wk2, wk3, wk4, wk5, wk6]
    
data1 = getAllWeeks(pigMyocardium,'MTR',0)
data2 = getAllWeeks(pigMyocardium,'T2',0) 
data3 = getAllWeeks(pigMyocardium,'MOLLI',0)   
data4 = getAllWeeks(pigMyocardium,'SHMOLLI',0)   
data5 = getAllWeeks(pigMyocardium,'SASHA',0)   
data6 = getAllWeeks(pigMyocardium,'IR',0)  
data7 = getAllWeeks(pigMyocardium,'T2',1)  
data8 = getAllWeeks(pigMyocardium,'MTR',1)  


    
data = data1 + data2 +  data3 + data4 + data5 + data6 + data7 + data8


MTRlabel = []
T2label = []
MOLLIlabel = []
SHMOLLIlabel = []
SASHAlabel = []
IRlabel = []
MTRexLabel = []
T2exLabel = []

for i in range(0,8):
    for j in range(0,6):
        
        if i == 0:
            MTRlabel.append(True)
        else:
            MTRlabel.append(False)
        if i == 1:
            T2label.append(True)
        else:
            T2label.append(False)
        if i == 2:
            MOLLIlabel.append(True)
        else:
            MOLLIlabel.append(False)
        if i == 3:
            SHMOLLIlabel.append(True)
        else:
            SHMOLLIlabel.append(False)
        if i == 4:
            SASHAlabel.append(True)
        else:
            SASHAlabel.append(False)
        if i == 5:
            IRlabel.append(True)
        else:
            IRlabel.append(False)
        if i == 6:
            T2exLabel.append(True)
        else:
            T2exLabel.append(False)
        if i == 7:
            MTRexLabel.append(True)
        else:
            MTRexLabel.append(False)
        



updatemenus = list([
    dict(
         
         buttons=list([   
            dict(label = 'MTR',
                 method = 'update',
                 args = [{'visible': MTRlabel},
                         {'title': '<b>Distribution of MTR values over six weeks</b> <br> <i>Prefixation: Wk1 to Wk3 | Postfixation: Wk4 to Wk6</i>',
                          'yaxis.range': (20,50),
                          'annotations': []}]),
             dict(label = 'MTR (w/o outliers)',
                 method = 'update',
                 args = [{'visible': MTRexLabel},
                         {'title': '<b>Distribution of MTR values over six weeks <i>after removing outliers</i></b> <br> <i>Prefixation: Wk1 to Wk3 | Postfixation: Wk4 to Wk6</i>',
                          'yaxis.range': (20,50),
                          'annotations': []}]),
            dict(label = 'T2',
                 method = 'update',
                 args = [{'visible': T2label},
                         {'title': '<b>Distribution of T2 values over six weeks</b> <br> <i>Prefixation: Wk1 to Wk3 | Postfixation: Wk4 to Wk6</i>',
                          'yaxis.range': (53,90),
                          'annotations': []}]),
             dict(label = 'T2 (w/o outliers)',
                 method = 'update',
                 args = [{'visible': T2exLabel},
                         {'title': '<b>Distribution of T2 values over six weeks <i>after removing outliers</i></b> <br> <i>Prefixation: Wk1 to Wk3 | Postfixation: Wk4 to Wk6</i>',
                          'yaxis.range': (53,90),
                          'annotations': []}]),
            dict(label = 'MOLLI',
                 method = 'update',
                 args = [{'visible': MOLLIlabel},
                         {'title': '<b>Distribution of T1 values by <i>MOLLI</i> over six weeks</b> <br> <i>Prefixation: Wk1 to Wk3 | Postfixation: Wk4 to Wk6</i>',
                          'yaxis.range': (400,1300),
                          'annotations': []}]),
            dict(label = 'SHMOLLI',
                 method = 'update',
                 args = [{'visible': SHMOLLIlabel},
                         {'title': '<b>Distribution of T1 values by <i>SHMOLLI</i> over six weeks</b> <br> <i>Prefixation: Wk1 to Wk3 | Postfixation: Wk4 to Wk6</i>',
                          'yaxis.range': (400,1300),
                          'annotations': []}]),
            dict(label = 'SASHA',
                 method = 'update',
                 args = [{'visible': SASHAlabel},
                         {'title': '<b>Distribution of T1 values by <i>SASHA</i> over six weeks</b> <br> <i>Prefixation: Wk1 to Wk3 | Postfixation: Wk4 to Wk6</i>',
                          'yaxis.range': (400,1300),
                          'annotations': []}]),
             dict(label = 'IR',
                 method = 'update',
                 args = [{'visible': IRlabel},
                         {'title': '<b>Distribution of T1 values by <i>gold standard IR</i> over six weeks</b> <br> <i>Prefixation: Wk1 to Wk3 | Postfixation: Wk4 to Wk6</i>',
                          'yaxis.range': (400,1300),
                          'annotations': []}])
        ]),
        type = 'buttons',
        active= -1,
        
        
    )
])

layout = go.Layout(
    width=990,
    height=600,
    showlegend=False, title =  'Please select a method (from the buttons on the left) to start interactive display.')

layout['updatemenus'] = updatemenus



fig = dict(data=data, layout=layout)
print('Click on me and hit [^] button (top toolbar) to display the code [1].')

# ----------------------------------------------------------------------------------------- NOTE
# You can regenerate this figure if your plotly credentials are set as described above. 
# Just uncomment the last line (remove the #) and hit <Ctrl + Enter>
# ----------------------------------------------------------------------------------------- 

#py.iplot(fig, filename='update_dropdown')

Click on me and hit [^] button (top toolbar) to display the code [1].


## <font color="Teal"> Figure 1: Comparing T1 mapping methods on one figure </font>
***
<i class="fa fa-info-circle" style="font-size: 1.5em;"></i> &nbsp; You can use legend on the right of the figure to show/hide traces. Double click on legend to isolate one trace, double click to reset.

***

In [117]:
IFrame('//plot.ly/~agkar/138.embed', width=1000, height=600)

<i class="fa fa-sort-down" style="font-size: 1.5em;"></i>

In [88]:
# ----------------------------------------------------------------------------------------- NOTE
# You can regenerate this figure if your plotly credentials are set as described above. 
# Make sure that you've run hidden code [1] before executing this cell. 
# Just uncomment the last line (remove the #) and hit <Ctrl + Enter>
# ----------------------------------------------------------------------------------------- 

hr_iter = 5; # Iterator for hearts 

# This function returns mean T1 for given week (wk) averaged over hearts.
def meanOnWeek(pigMyocardium,wk,method):
    a = [];
    for hr_iter in range(0,6):
          exec('a.append(np.mean(np.array(pigMyocardium[0,hr_iter].Week' + str(wk) + '[0,0].' + method + 'vec)))')
    return  [np.mean(a), np.std(a)]

# This function returns mean & std (over hearts) for each week. 
def overWeek(pigMyocardium,method):
    a = []
    for j in range(0,6):
        a.append(meanOnWeek(pigMyocardium,j+1,method))
    return a
    

# Figure 1 

overWeek_IR = overWeek(pigMyocardium,'IR')
overWeek_MOLLI = overWeek(pigMyocardium,'MOLLI')
overWeek_SHMOLLI = overWeek(pigMyocardium,'SHMOLLI')
overWeek_SASHA = overWeek(pigMyocardium,'SASHA')

irWeekly = go.Scatter(
    x=[1, 2, 3, 4, 5, 6],
    y=[overWeek_IR[0][0],overWeek_IR[1][0],overWeek_IR[2][0],overWeek_IR[3][0],overWeek_IR[4][0],overWeek_IR[5][0]],
    name='IR',
    marker={'size': "12"},
    error_y=dict(
            type='data',
            array=[overWeek_IR[0][1],overWeek_IR[1][1],overWeek_IR[2][1],overWeek_IR[3][1],overWeek_IR[4][1],overWeek_IR[5][1]],
            visible=True
        ))
molliWeekly = go.Scatter(
    x=[1, 2, 3, 4, 5, 6],
    y=[overWeek_MOLLI[0][0],overWeek_MOLLI[1][0],overWeek_MOLLI[2][0],overWeek_MOLLI[3][0],overWeek_MOLLI[4][0],overWeek_MOLLI[5][0]],
    name='MOLLI',
    marker={'size': "12"},
    error_y=dict(
            type='data',
            array=[overWeek_MOLLI[0][1],overWeek_MOLLI[1][1],overWeek_MOLLI[2][1],overWeek_MOLLI[3][1],overWeek_MOLLI[4][1],overWeek_MOLLI[5][1]],
            visible=True
        ))
    
    
shmolliWeekly = go.Scatter(
    x=[1, 2, 3, 4, 5, 6],
    y=[overWeek_SHMOLLI[0][0],overWeek_SHMOLLI[1][0],overWeek_SHMOLLI[2][0],overWeek_SHMOLLI[3][0],overWeek_SHMOLLI[4][0],overWeek_SHMOLLI[5][0]],
    name='SHMOLLI',
    marker={'size': "12"},
    error_y=dict(
            type='data',
            array =[overWeek_SHMOLLI[0][1],overWeek_SHMOLLI[1][1],overWeek_SHMOLLI[2][1],overWeek_SHMOLLI[3][1],overWeek_SHMOLLI[4][1],overWeek_SHMOLLI[5][1]],
            visible=True
        ))
    
    
    
    
sashaWeekly = go.Scatter(
    x=[1, 2, 3, 4, 5, 6],
    y=[overWeek_SASHA[0][0],overWeek_SASHA[1][0],overWeek_SASHA[2][0],overWeek_SASHA[3][0],overWeek_SASHA[4][0],overWeek_SASHA[5][0]],
    name='SASHA',
    marker={'size': "12"},
    error_y=dict(
            type='data',
            array=[overWeek_SASHA[0][1],overWeek_SASHA[1][1],overWeek_SASHA[2][1],overWeek_SASHA[3][1],overWeek_SASHA[4][1],overWeek_SASHA[5][1]],
            visible=True
        ))

layout=go.Layout(title="Figure 1: Evolution of T1 over time for all four sequences, before (lavender) and after (turquoise) fixation.", xaxis={'title':'Week'}, yaxis={'title':'T1 values (ms)'})
data = [molliWeekly, shmolliWeekly, sashaWeekly, irWeekly]
layout.update(dict(shapes = [
        # 1st highlight during Feb 4 - Feb 6
        {
            'type': 'rect',
            # x-reference is assigned to the x-values
            'xref': 'x',
            # y-reference is assigned to the plot paper [0,1]
            'yref': 'y',
            'x0': '0.95',
            'y0': 365,
            'x1': '3.05',
            'y1': 1300,
            'fillcolor': '#c2cae8',
            'opacity': 0.3,
            'line': {
                'width': 0,
            }
            
        },
    
        {
            'type': 'rect',
            # x-reference is assigned to the x-values
            'xref': 'x',
            # y-reference is assigned to the plot paper [0,1]
            'yref': 'y',
            'x0': '3.95',
            'y0': 365,
            'x1': '6.05',
            'y1': 1300,
            'fillcolor': '#45f0df',
            'opacity': 0.15,
            'line': {
                'width': 0,
            }
            
        }
    
    
        ]
        ))

print('Click on me & hit [^] button (top toolbar) to display the code [2].')

# ----------------------------------------------------------------------------------------- NOTE
# You can regenerate this figure if your plotly credentials are set as described above. 
# Make sure that you've run hidden code [1] before executing this cell. 
# Just uncomment the last line (remove the #) and hit <Ctrl + Enter>
# ----------------------------------------------------------------------------------------- 

#py.iplot(go.Figure(data=data, layout=layout), filename='Figure1')




Click on me & hit [^] button (top toolbar) to display the code [2].


## <font color="Teal"> Figure 2: Comparison of _T1 Bias_ between MOLLI, SHMOLLI and SASHA before and after fixation. </font>
***

T1 bias (ms) values are obtained by subtracting gold standard IR maps from those acquired by MOLLI, SHMOLLI and SASHA.

***

In [118]:
IFrame('//plot.ly/~agkar/140.embed', width=1000, height=600)

<i class="fa fa-sort-down" style="font-size: 1.5em;"></i>

In [89]:
# ----------------------------------------------------------------------------------------- NOTE
# You can regenerate this figure if your plotly credentials are set as described above. 
# Make sure that you've run hidden code [1 and 2] before executing this cell. 
# Just uncomment the last line (remove the #) and hit <Ctrl + Enter>
# ----------------------------------------------------------------------------------------- 

b4Fix_IR = np.array([overWeek_IR[0][0],overWeek_IR[1][0],overWeek_IR[2][0]])
l8rFix_IR = np.array([overWeek_IR[3][0],overWeek_IR[4][0],overWeek_IR[5][0]])

b4Fix_MOLLI = np.array([overWeek_MOLLI[0][0],overWeek_MOLLI[1][0],overWeek_MOLLI[2][0]])
l8rFix_MOLLI = np.array([overWeek_MOLLI[3][0],overWeek_MOLLI[4][0],overWeek_MOLLI[5][0]])

b4Fix_SHMOLLI = np.array([overWeek_SHMOLLI[0][0],overWeek_SHMOLLI[1][0],overWeek_SHMOLLI[2][0]])
l8rFix_SHMOLLI = np.array([overWeek_SHMOLLI[3][0],overWeek_SHMOLLI[4][0],overWeek_SHMOLLI[5][0]])

b4Fix_SASHA = np.array([overWeek_SASHA[0][0],overWeek_SASHA[1][0],overWeek_SASHA[2][0]])
l8rFix_SASHA = np.array([overWeek_SASHA[3][0],overWeek_SASHA[4][0],overWeek_SASHA[5][0]])

trace1 = go.Bar(
    x=['MOLLI', 'SHMOLLI', 'SASHA'],
    y=[np.mean(b4Fix_MOLLI-b4Fix_IR), np.mean(b4Fix_SHMOLLI-b4Fix_IR), np.mean(b4Fix_SASHA-b4Fix_IR)],
    name='Before Fixation',
    error_y=dict(
        type='data',
        array=[np.std(b4Fix_MOLLI-b4Fix_IR), np.std(b4Fix_SHMOLLI-b4Fix_IR), np.std(b4Fix_SASHA-b4Fix_IR)],
        visible=True
    )
)
trace2 = go.Bar(
    x=['MOLLI', 'SHMOLLI', 'SASHA'],
    y=[np.mean(l8rFix_MOLLI-l8rFix_IR), np.mean(l8rFix_SHMOLLI-l8rFix_IR), np.mean(l8rFix_SASHA-l8rFix_IR)],
    name='After Fixation',
    error_y=dict(
        type='data',
        array=[np.std(l8rFix_MOLLI-l8rFix_IR), np.std(l8rFix_SHMOLLI-l8rFix_IR), np.std(l8rFix_SASHA-l8rFix_IR)],
        visible=True
    )
)
data = [trace1, trace2]
layout=go.Layout(title="Figure 1: Evolution of T1 over time for all four sequences, before (lavender) and after (turquoise) fixation.", xaxis={'title':'Week'}, yaxis={'title':'T1 values (ms)'})

layout = go.Layout(
    barmode='group',
    title='Figure 2: Before-after fixation comparisons of MOLLI, SHMOLLI and SASHA T1 bias.',
    xaxis={'title':'Reference: IR-TSE'},
    yaxis={'title':'T1 (ms) bias'}
    
)
fig = go.Figure(data=data, layout=layout)

print('Click on me & hit [^] button (top toolbar) to display the code [3].')

# ----------------------------------------------------------------------------------------- NOTE
# You can regenerate this figure if your plotly credentials are set as described above. 
# Make sure that you've run hidden code [1 and 2] before executing this cell. 
# Just uncomment the last line (remove the #) and hit <Ctrl + Enter>
# ----------------------------------------------------------------------------------------- 


#py.iplot(go.Figure(data=data, layout=layout), filename='Figure2_pigExVivo')

    


Click on me & hit [^] button (top toolbar) to display the code [3].


## <font color="Teal"> Figure 3: Association between _T1 Bias_ and MTR and T2 effects based on phantom acquisitions. </font>
***

T1 bias (ms) values are obtained by subtracting gold standard IR maps from those acquired by MOLLI, SHMOLLI and SASHA.

***

In [119]:
IFrame('//plot.ly/~agkar/142.embed', width=1000, height=450)


<i class="fa fa-sort-down" style="font-size: 1.5em;"></i>

In [90]:
# ----------------------------------------------------------------------------------------- NOTE
# You can regenerate this figure if your plotly credentials are set as described above. 
# Make sure that you've run hidden code [1, 2 and 3] before executing this cell. 
# Just uncomment the last line (remove the #) and hit <Ctrl + Enter>
# -----------------------------------------------------------------------------------------

from numpy import arange,array,ones
from scipy import stats
from plotly import tools
import numpy.matlib

# Conventional Pearson's Correlation 

def corPlot(xData,yData,colorPoint,colorLine,dataName,xRefName):
# Creating the dataset, and generating the plot

    slope, intercept, r_value, p_value, std_err = stats.linregress(xData,yData)
    line = slope*xData+intercept
    a = 'r: ' + str(round(r_value,4)) + '<br>' + 'p: ' + str(round(p_value,4))
    aNot = np.matlib.repmat(a,1,np.size(yData));
    aNot = np.squeeze(aNot)
    
    trace1 = go.Scatter(
                      x=xData,
                      y=yData,
                      mode='markers',
                      marker=go.Marker(color=colorPoint, size = '10'),
                      name= dataName + ' vs ' + xRefName,
                      legendgroup=dataName + ' vs ' + xRefName,
                      )

    trace2 = go.Scatter(
                      x=xData,
                      y=line,
                      mode='lines',
                      marker=go.Marker(color= colorLine),
                      name=dataName + ' vs ' + xRefName,
                      legendgroup=dataName + ' vs ' + xRefName,
                      text= aNot,
                      hoverinfo= 'text',
                      showlegend = False
                      )
    


    return trace1, trace2

def meanOnPhantom(phantomData,method):
    a = [];
    for roi_iter in range(0,7):
          exec('a.append(np.mean(np.array(phantomData[0,roi_iter].' +  method + 'vec)))')
    return  np.array(a)




phan_IR = meanOnPhantom(phantomData,'IR')
phan_MTR = meanOnPhantom(phantomData,'MTR') 
phan_T2 = meanOnPhantom(phantomData,'T2')
phan_MOLLI = meanOnPhantom(phantomData,'MOLLI') - phan_IR
phan_SHMOLLI = meanOnPhantom(phantomData,'SHMOLLI') - phan_IR
phan_SASHA = meanOnPhantom(phantomData,'SASHA') - phan_IR



sashaColor = 'rgb(44,160,44)'
molliColor = 'rgb(31,119,180)'
shmolliColor = 'rgb(255,127,14)'
irColor = 'rgb(214,39,40)'
colorLine = 'rgb(31, 119, 180)'

trace1, trace2 = corPlot(phan_T2,phan_MOLLI,molliColor,molliColor,'MOLLI','T2')
trace3, trace4= corPlot(phan_T2,phan_SHMOLLI,shmolliColor,shmolliColor,'SHMOLLI','T2')
trace5, trace6 = corPlot(phan_T2,phan_SASHA,sashaColor,sashaColor,'SASHA','T2')

trace7, trace8 = corPlot(phan_MTR,phan_MOLLI,molliColor,molliColor,'MOLLI','MTR')
trace9, trace10= corPlot(phan_MTR,phan_SHMOLLI,shmolliColor,shmolliColor,'SHMOLLI','MTR')
trace11, trace12 = corPlot(phan_MTR,phan_SASHA,sashaColor,sashaColor,'SASHA','MTR')



fig = tools.make_subplots(rows=1, cols=2, 
                          shared_xaxes=True, shared_yaxes=True,
                          vertical_spacing=0.01,subplot_titles=('a', 'b'), print_grid=False)

fig['layout']['xaxis1'].update(title='T2 (ms)')
fig['layout']['xaxis2'].update(title='MTR(ms)')



fig['layout']['yaxis1'].update(title='T1 (ms) bias')


fig.append_trace(trace5, 1, 1)
fig.append_trace(trace6, 1, 1)
fig.append_trace(trace3, 1, 1)
fig.append_trace(trace4, 1, 1)
fig.append_trace(trace1, 1, 1)
fig.append_trace(trace2, 1, 1)

fig.append_trace(trace7, 1, 2)
fig.append_trace(trace8, 1, 2)
fig.append_trace(trace9, 1, 2)
fig.append_trace(trace10, 1, 2)
fig.append_trace(trace11, 1, 2)
fig.append_trace(trace12, 1, 2)


fig['layout'].update(height=450, width=1000,title='Figure 3: Correlation plots for (phantom data) MOLLI, SHMOLLI and SASHA T1 bias with a) T2 and b) MTR.')

print('Click on me & hit [^] button (top toolbar) to display the code [4].')

# ----------------------------------------------------------------------------------------- NOTE
# You can regenerate this figure if your plotly credentials are set as described above. 
# Make sure that you've run hidden code [1, 2 and 3] before executing this cell. 
# Just uncomment the last line (remove the #) and hit <Ctrl + Enter>
# -----------------------------------------------------------------------------------------

#py.iplot(fig, filename='Figure3')

Click on me & hit [^] button (top toolbar) to display the code [4].


## <font color="Teal"> Figure 4: Association between _T1 Bias_ and T2</font>
***

Explain.

***

In [120]:
IFrame('//plot.ly/~agkar/144.embed', width=1000, height=800)

<i class="fa fa-sort-down" style="font-size: 1.5em;"></i>

In [99]:
# ----------------------------------------------------------------------------------------- NOTE
# You can regenerate this figure if your plotly credentials are set as described above. 
# Make sure that you've run hidden code [1, 2, 3 and 4] before executing this cell. 
# Just uncomment the last line (remove the #) and hit <Ctrl + Enter>
# -----------------------------------------------------------------------------------------

from collections import namedtuple
import string
from scipy import signal
from scipy.stats import t as tstat
import math


sashaColor = 'rgb(44,160,44)'
molliColor = 'rgb(31,119,180)'
shmolliColor = 'rgb(255,127,14)'
irColor = 'rgb(214,39,40)'
colorLine = 'rgb(31, 119, 180)'

sashaFill = 'rgba(44,160,44,0.3)'
molliFill = 'rgba(31,119,180,0.3)'
shmolliFill = 'rgba(255,127,14,0.3)'
insigFill = 'rgba(255,0,0,0.3)'


mat = loadmat('exvivoCorrelation.mat', struct_as_record=False)

# Split .mat data 
ir = mat['IR']
mtr = mat['MTR']
t2 = mat['T2']
molli =  mat['MOLLI']
shmolli = mat['SHMOLLI']
sasha = mat['SASHA']
preLab = np.array(mat['preLab'])
postLab = np.array(mat['postLab'])
pooLab = np.array(mat['pooLab'])


# Make namedtuple definitions

T2stat = namedtuple("T2stat", "elX elY CI outliers pval")
MTRstat = namedtuple("MTRstat", "elX elY CI outliers pval")

prefix = namedtuple("prefix", "Bias T2stat MTRstat")
postfix = namedtuple("postfix", "Bias T2stat MTRstat")
pool = namedtuple("pool", "Bias T2stat MTRstat")

molliDef = namedtuple("molli", "prefix postfix pool")
shmolliDef = namedtuple("shmolli", "prefix postfix pool")
sashaDef = namedtuple("sasha", "prefix postfix pool")

mtrDef = namedtuple("mtr", "prefix postfix pool")
t2Def = namedtuple("t2", "prefix postfix pool")
irDef = namedtuple("ir", "prefix postfix")


def convertMat2Tuple(method):
    # Prefixation
    exec('t2str = T2stat(np.array(' + method + '[0,0].prefix[0,0].T2[0,0].elX),np.array(' + method + '[0,0].prefix[0,0].T2[0,0].elY),np.array(' + method+ '[0,0].prefix[0,0].T2[0,0].CI),np.array(' + method + '[0,0].prefix[0,0].T2[0,0].outlierBin),np.array(' + method +  '[0,0].prefix[0,0].T2[0,0].pval))')
    exec('mtrstr = MTRstat(np.array(' + method + '[0,0].prefix[0,0].MTR[0,0].elX),np.array(' + method + '[0,0].prefix[0,0].MTR[0,0].elY),np.array(' + method+ '[0,0].prefix[0,0].MTR[0,0].CI),np.array(' + method + '[0,0].prefix[0,0].MTR[0,0].outlierBin),np.array(' + method +  '[0,0].prefix[0,0].MTR[0,0].pval))')
    exec('prestr = prefix(np.array(' + method + '[0,0].prefix[0,0].Bias),t2str,mtrstr)')
    # Postfixation
    exec('t2str = T2stat(np.array(' + method + '[0,0].postfix[0,0].T2[0,0].elX),np.array(' + method + '[0,0].postfix[0,0].T2[0,0].elY),np.array(' + method+ '[0,0].postfix[0,0].T2[0,0].CI),np.array(' + method + '[0,0].postfix[0,0].T2[0,0].outlierBin),np.array(' + method +  '[0,0].postfix[0,0].T2[0,0].pval))')
    exec('mtrstr = MTRstat(np.array(' + method + '[0,0].postfix[0,0].MTR[0,0].elX),np.array(' + method + '[0,0].postfix[0,0].MTR[0,0].elY),np.array(' + method+ '[0,0].postfix[0,0].MTR[0,0].CI),np.array(' + method + '[0,0].postfix[0,0].MTR[0,0].outlierBin),np.array(' + method +  '[0,0].postfix[0,0].MTR[0,0].pval))')
    exec('poststr = postfix(np.array(' + method + '[0,0].postfix[0,0].Bias),t2str,mtrstr)')
    # Pool
    exec('t2str = T2stat(np.array(' + method + '[0,0].pool[0,0].T2[0,0].elX),np.array(' + method + '[0,0].pool[0,0].T2[0,0].elY),np.array(' + method+ '[0,0].pool[0,0].T2[0,0].CI),np.array(' + method + '[0,0].pool[0,0].T2[0,0].outlierBin),np.array(' + method +  '[0,0].pool[0,0].T2[0,0].pval))')
    exec('mtrstr = MTRstat(np.array(' + method + '[0,0].pool[0,0].MTR[0,0].elX),np.array(' + method + '[0,0].pool[0,0].MTR[0,0].elY),np.array(' + method+ '[0,0].pool[0,0].MTR[0,0].CI),np.array(' + method + '[0,0].pool[0,0].MTR[0,0].outlierBin),np.array(' + method +  '[0,0].pool[0,0].MTR[0,0].pval))')
    exec('poolstr = pool(np.array(' + method + '[0,0].pool[0,0].Bias),t2str,mtrstr)')
    exec( 'out ='+ method + 'Def(prestr,poststr,poolstr)')
    return out 

# Get data for MOLLI, SHMOLLI and SASHA 

MOLLI = convertMat2Tuple('molli')
SHMOLLI = convertMat2Tuple('shmolli')
SASHA = convertMat2Tuple('sasha')

# IR, MTR and T2 do not have T2stat and MTRstat, therefore assingment cannot be done using convertMat2Tuple

MTR = mtrDef(np.array(mtr[0,0].prefix), np.array(mtr[0,0].postfix), np.array(mtr[0,0].pool))
T2 = t2Def(np.array(t2[0,0].prefix), np.array(t2[0,0].postfix), np.array(t2[0,0].pool))
IR = irDef(np.array(ir[0,0].prefix), np.array(ir[0,0].postfix))


def Pearson(X,Y,N):
    # N is passed to this function because in the presence of bivariate outliers, still the total n is used.
    # For details, please see https://www.frontiersin.org/articles/10.3389/fpsyg.2012.00606/full 
    X = (X)
    Y = (Y)
    n = N
    Xdetrend = np.transpose(signal.detrend(X,type = 'constant'))
    Ydetrend = np.transpose(signal.detrend(Y,type = 'constant'))
    r = sum(Xdetrend*Ydetrend)/math.sqrt(sum(Xdetrend**2)*sum((Ydetrend)**2))
    tp = r*math.sqrt((n-2)/(1-r**2))
    pval = 2*tstat.cdf(-abs(tp),n-2)
    return r, pval





def corPlotOutlier(*args):
    xData = args[0]
    yData = args[1]
    colorPoint = args[2]
    colorFill = args[3]
    dataName = args[4]
    
    hvr = args[5]
    elX = args[6]
    elY = args[7]
    CIlow = args[8]
    CIup = args[9]
    
    # Linregress function is called either case to draw best fit line. 
    # However, in case of outliers although r_value is going to be the same, pval will computed differently.
    # In this case, using CI for estimating significance is more appropriate. 
    
    if len(args) > 10: # If outliers are detected
        nout = len(args[10]) # This is the number of outliers 
        N = len(xData) 
        slope, intercept, _, _, std_err = stats.linregress(xData,yData)
        r_value, p_value = Pearson(xData,yData,N+nout)
    else: # If outlier is not detected
        slope, intercept, r_value, p_value, std_err = stats.linregress(xData,yData)
    
    line = slope*xData+intercept
    if intercept<0:
        aNot = '<b>' + dataName + '</b>'  + ' | ' + 'y = ' + str(round(slope,1)) + 'x'  + str(round(intercept,1)) + '<br>' + 'r: ' + str(round(r_value,2)) + ' | ' + 'p: ' + str(round(p_value,2)) + ' | CI: [' + str(round(CIlow,2)) + ' ' + str(round(CIup,2)) + ']' 
    if intercept>0:
        aNot = '<b>'+ dataName + '</b>'  + ' | '+  'y = ' + str(round(slope,1)) + 'x+'  + str(round(intercept,1)) + '<br>' + 'r: ' + str(round(r_value,2)) + ' | ' + 'p: ' + str(round(p_value,2)) + ' | CI: [' + str(round(CIlow,2)) + ' ' + str(round(CIup,2)) + ']' 

    if not((CIup>0 and CIlow>0) or (CIup<0 and CIlow<0)):
        colorFill = insigFill
   
    
    trace1 = go.Scatter(
                      x= xData,
                      y= yData,
                      mode='markers',
                      marker=go.Marker(color=colorPoint, size = '10',line=dict(width=2, color = 'rgb(256,256,256)')),
                      opacity=0.7,
                      xaxis='x1',
                      yaxis='y2',
                      name = dataName,
                      legendgroup =  dataName,
                      text= hvr,
                      hoverinfo= 'all',
                      showlegend = True
                      )

    trace2 = go.Scatter(
                      x=xData,
                      y=line,
                      mode='lines',
                      xaxis='x1',
                      yaxis='y2',
                      legendgroup =  dataName,
                      marker=go.Marker(color= colorPoint),
                      name= dataName,
                      hoverinfo= 'none',
                      showlegend = False
                      )
    trace3 = go.Scatter(
                      x= elX,
                      y= elY,
                      fill= 'toself',
                      fillcolor = colorFill,
                      xaxis='x1',
                      yaxis='y2',
                      name = string.upper(dataName.split()[1]),
                      marker=go.Marker(color= 'rgba(0,0,0,0.001)'),
                      hoverinfo= 'none',
                      legendgroup =  dataName,
                      showlegend = False
                      )
    if len(args) > 10:
        trace4 = go.Scatter(
                          x= args[10],
                          y= args[11],
                          mode='markers',
                          #marker=go.Marker(color='rgb(107,174,214)', size = '10',line=dict(width=2, color = 'rgb(107,174,214)')),
                          xaxis='x1',
                          yaxis='y2',
                          name = string.upper(dataName.split()[1]),
                          text= args[12],
                          hoverinfo= 'all',
                          legendgroup =  dataName,
                          showlegend = False
                          )
    else:
        trace4 = []
    
    return trace1, trace2, trace3, trace4, aNot
    
    
def getExcluded(inp,fixName,statName, pointColor, fillColor, descrName):
    # This function can be called only if there are outliers 
    xData = []
    yData = []
    outX = []
    outY = []
    hvrOut = []
    hvr = []
    nm = statName + 'stat'
    stat = getattr(getattr(inp,fixName),nm)
    bias = np.squeeze(getattr(getattr(inp,fixName),'Bias'))
    outIdx = np.squeeze(stat.outliers) 
    elX = np.transpose(np.squeeze(stat.elX))
    elY = np.transpose(np.squeeze(stat.elY))
    CIup = max(np.squeeze(stat.CI))
    CIlow = min(np.squeeze(stat.CI))
    exec('xDat =' + statName + '.' + fixName)
    xDat = np.squeeze(xDat)
    
    for it in range(len(outIdx)):
        if outIdx[it] == 0:
            yData.append(bias[it])
            xData.append(xDat[it])
            if fixName is 'prefix':
                hvr.append('Week: ' + (str([int(s) for s in str(preLab[it,1]) if s.isdigit()])) + ' | ' + 'Heart: ' + (str([int(s) for s in str(preLab[it,0]) if s.isdigit()])))
            elif fixName is 'postfix':
                hvr.append('Week: ' + (str([int(s) for s in str(postLab[it,1]) if s.isdigit()])) + ' | ' + 'Heart: ' + (str([int(s) for s in str(postLab[it,0]) if s.isdigit()])))
            elif fixName is 'pool':
                hvr.append('Week: ' + (str([int(s) for s in str(pooLab[it,1]) if s.isdigit()])) + ' | ' + 'Heart: ' + (str([int(s) for s in str(pooLab[it,0]) if s.isdigit()])))    
        elif outIdx[it] == 1: # This means that the points is an outlier
            outY.append(bias[it])
            outX.append(xDat[it])
            if fixName is 'prefix':
                hvrOut.append(string.upper(descrName.split()[1]) + ' Outlier' + '<br>'+'Week: ' + (str([int(s) for s in str(preLab[it,1]) if s.isdigit()])) + ' | ' + 'Heart: ' + (str([int(s) for s in str(preLab[it,0]) if s.isdigit()])))
            elif fixName is 'postfix':
                hvrOut.append(string.upper(descrName.split()[1]) + ' Outlier' + '<br>'+'Week: ' + (str([int(s) for s in str(postLab[it,1]) if s.isdigit()])) + ' | ' + 'Heart: ' + (str([int(s) for s in str(postLab[it,0]) if s.isdigit()])))
            elif fixName is 'pool':
                hvrOut.append(string.upper(descrName.split()[1]) + ' Outlier' + '<br>'+'Week: ' + (str([int(s) for s in str(pooLab[it,1]) if s.isdigit()])) + ' | ' + 'Heart: ' + (str([int(s) for s in str(pooLab[it,0]) if s.isdigit()])))
                
    xData = np.transpose(xData) # Transpose is required after appending data, otherwise regress won't work properly
    yData = np.transpose(yData) 
    outX = np.transpose(outX)
    outY = np.transpose(outY)
    
  
    if not len(outX) == 0:
        out = corPlotOutlier(xData,yData,pointColor, fillColor,descrName,hvr, elX, elY, CIlow, CIup, outX, outY, hvrOut)
    else:
        out = corPlotOutlier(xData,yData,pointColor, fillColor,descrName,hvr, elX, elY, CIlow, CIup)
    
    return  out


def traceAppend(inPlot,fig,xAx,yAx):
    if not len(inPlot[3]) == 0:
        fig.append_trace(inPlot[2], xAx, yAx)
        fig.append_trace(inPlot[1], xAx, yAx)
        fig.append_trace(inPlot[0], xAx, yAx)
        fig.append_trace(inPlot[3], xAx, yAx)
    else:
        fig.append_trace(inPlot[2], xAx, yAx)
        fig.append_trace(inPlot[1], xAx, yAx)
        fig.append_trace(inPlot[0], xAx, yAx)
        
        

preMolli_t2  = getExcluded(MOLLI,'prefix','T2', molliColor, molliFill, 'pre MOLLI')
preShmolli_t2  = getExcluded(SHMOLLI,'prefix','T2', shmolliColor, shmolliFill, 'pre SHMOLLI')
preSasha_t2  = getExcluded(SASHA,'prefix','T2', sashaColor, sashaFill, 'pre SASHA')

postMolli_t2  = getExcluded(MOLLI,'postfix','T2', molliColor, molliFill, 'post MOLLI')
postShmolli_t2  = getExcluded(SHMOLLI,'postfix','T2', shmolliColor, shmolliFill, 'post SHMOLLI')
postSasha_t2  = getExcluded(SASHA,'postfix','T2', sashaColor, sashaFill, 'post SASHA')

poolMolli_t2  = getExcluded(MOLLI,'pool','T2', molliColor, molliFill, 'pool MOLLI')
poolShmolli_t2  = getExcluded(SHMOLLI,'pool','T2', shmolliColor, shmolliFill, 'pool SHMOLLI')
poolSasha_t2  = getExcluded(SASHA,'pool','T2', sashaColor, sashaFill, 'pool SASHA')


fig = tools.make_subplots(rows=2, cols=3, shared_xaxes=False, shared_yaxes=True, vertical_spacing=0.15,horizontal_spacing= 0.01, print_grid=False)

traceAppend(preMolli_t2,fig,1,1)
traceAppend(preShmolli_t2,fig,1,1)
traceAppend(preSasha_t2,fig,1,1)

traceAppend(postMolli_t2,fig,1,2)
traceAppend(postShmolli_t2,fig,1,2)
traceAppend(postSasha_t2,fig,1,2)

traceAppend(poolMolli_t2,fig,1,3)
traceAppend(poolShmolli_t2,fig,1,3)
traceAppend(poolSasha_t2,fig,1,3)

# Plotly follows raw-major order here. 
fig['layout']['xaxis1'].update(title='Prefixation T2 (ms)')
fig['layout']['xaxis5'].update(title='Postfixatoin T2 (ms)')
fig['layout']['xaxis6'].update(title='Pooled T2 (ms)')

fig['layout']['yaxis1'].update(title='T1 bias (ms)')

leg = dict(
        x= 0.1,
        y=-0.19,
        orientation="h",
        traceorder = "grouped",
        
        font=dict(
            family='sans-serif',
            size=8,
            color= '#FFFFFF'
        ),
        bgcolor='#173753',
        bordercolor='#FFFFFF',
        borderwidth=2
    )


fig['layout'].update(height=800, width=1000,title='Figure 4.1: Associations between T2 effects and T1 bias in explanted hearts.', legend= leg , hovermode = 'closest')

# Display robust correlation results START

fig['layout'].update(
    annotations=[
    # Sasha pre T2
    dict(
            x=2,
            y=4,
            xref= 'x4',
            yref= 'y2',
            text= preSasha_t2[4], #+ '<br> <br>' + preMolli_t2[4]+ '<br> <br>' + preShmolli_t2[4],
            showarrow=False,
            font=dict(
                family='Lucida Console, monospace',
                size=11,
                color='#ffffff'
            ),
            align='left',
            bordercolor='#c7c7c7',
            borderwidth=0,
            borderpad=0,
            bgcolor= sashaColor,
            opacity=0.6
      
        ),
        # Molli pre T2
        dict(
            x=1.9,
            y=2,
            xref= 'x4',
            yref= 'y2',
            text= preMolli_t2[4],
            showarrow=False,
            font=dict(
                family='Lucida Console, monospace',
                size=11,
                color='#ffffff'
            ),
            align='left',
            bordercolor='#c7c7c7',
            borderwidth=0,
            borderpad=0,
            bgcolor= molliColor,
            opacity=0.6
      
        ),
        #Shmolli pre T2
        dict(
            x=1.7,
            y=0,
            xref= 'x4',
            yref= 'y2',
            text= preShmolli_t2[4],
            showarrow=False,
            font=dict(
                family='Lucida Console, monospace',
                size=11,
                color='#ffffff'
            ),
            align='left',
            bordercolor='#c7c7c7',
            borderwidth=0,
            borderpad=0,
            bgcolor= shmolliColor,
            opacity=0.6
      
        ),
                
        # Sasha post T2
    dict(
            x=1.86,
            y=4,
            xref= 'x5',
            yref= 'y2',
            text= postSasha_t2[4], #+ '<br> <br>' + preMolli_t2[4]+ '<br> <br>' + preShmolli_t2[4],
            showarrow=False,
            font=dict(
                family='Lucida Console, monospace',
                size=11,
                color='#ffffff'
            ),
            align='left',
            bordercolor='#c7c7c7',
            borderwidth=0,
            borderpad=0,
            bgcolor= sashaColor,
            opacity=0.6
      
        ),
        # Molli post T2
        dict(
            x=2,
            y=2,
            xref= 'x5',
            yref= 'y2',
            text= postMolli_t2[4],
            showarrow=False,
            font=dict(
                family='Lucida Console, monospace',
                size=11,
                color='#ffffff'
            ),
            align='left',
            bordercolor='#c7c7c7',
            borderwidth=0,
            borderpad=0,
            bgcolor= molliColor,
            opacity=0.6
      
        ),
        #Shmolli post T2
        dict(
            x=2,
            y=0,
            xref= 'x5',
            yref= 'y2',
            text= postShmolli_t2[4],
            showarrow=False,
            font=dict(
                family='Lucida Console, monospace',
                size=11,
                color='#ffffff'
            ),
            align='left',
            bordercolor='#c7c7c7',
            borderwidth=0,
            borderpad=0,
            bgcolor= shmolliColor,
            opacity=0.6
      
        ),
                
        # Sasha pool T2
    dict(
            x=1.8,
            y=4,
            xref= 'x6',
            yref= 'y2',
            text= poolSasha_t2[4], #+ '<br> <br>' + preMolli_t2[4]+ '<br> <br>' + preShmolli_t2[4],
            showarrow=False,
            font=dict(
                family='Lucida Console, monospace',
                size=11,
                color='#ffffff'
            ),
            align='left',
            bordercolor='#c7c7c7',
            borderwidth=0,
            borderpad=0,
            bgcolor= sashaColor,
            opacity=0.6
      
        ),
        # Molli pool T2
        dict(
            x=2,
            y=2,
            xref= 'x6',
            yref= 'y2',
            text= poolMolli_t2[4],
            showarrow=False,
            font=dict(
                family='Lucida Console, monospace',
                size=11,
                color='#ffffff'
            ),
            align='left',
            bordercolor='#c7c7c7',
            borderwidth=0,
            borderpad=0,
            bgcolor= molliColor,
            opacity=0.6
      
        ),
        #Shmolli pool T2
        dict(
            x=2,
            y=0,
            xref= 'x6',
            yref= 'y2',
            text= poolShmolli_t2[4],
            showarrow=False,
            font=dict(
                family='Lucida Console, monospace',
                size=11,
                color='#ffffff'
            ),
            align='left',
            bordercolor='#c7c7c7',
            borderwidth=0,
            borderpad=0,
            bgcolor= shmolliColor,
            opacity=0.6
      
        )
]
        )

# Display robust correlation results END 

fig['layout']['xaxis1'].update(
side='bottom',
position = -1)
fig['layout']['xaxis2'].update(
side='bottom',
position = -1)
fig['layout']['xaxis3'].update(
side='bottom',
position = -1)

fig['layout']['yaxis1'].update(
        domain = [0, 0.8] )
fig['layout']['yaxis2'].update(autorange=False,
        showgrid=False,
        zeroline=False,
        showline=False,
        autotick=True,
        ticks='',
        showticklabels=False,
        domain = [0.8, 1]
        )

fig['layout']['yaxis3'].update(
        domain = [0, 0.8] )

fig['layout']['yaxis4'].update(autorange=False,
        showgrid=False,
        zeroline=False,
        showline=False,
        autotick=True,
        ticks='',
        showticklabels=False,
        domain = [0.8, 1],
        range = [0, 10]  )

fig['layout']['yaxis5'].update(
        
        domain = [0, 0.8] )

fig['layout']['yaxis6'].update(autorange=False,
        showgrid=False,
        zeroline=False,
        showline=False,
        autotick=True,
        ticks='',
        showticklabels= False,
        domain = [0.8, 1],
       range = [0, 10] )

fig['layout']['xaxis4'].update(autorange=False,
        showgrid=False,
        zeroline=False,
        showline=False,
        autotick=True,
        ticks='',
        showticklabels=False                   
        )

fig['layout']['xaxis5'].update(autorange=False,
        showgrid=False,
        zeroline=False,
        showline=False,
        autotick=True,
        ticks='',
        showticklabels=False
        )
    
fig['layout']['xaxis6'].update(autorange=False,
        showgrid=False,
        zeroline=False,
        showline=False,
        autotick=True,
        ticks='',
        showticklabels=False                       
        )

config={'showLink': False}

print('Click on me & hit [^] button (top toolbar) to display the code [5].')

# ----------------------------------------------------------------------------------------- NOTE
# You can regenerate this figure if your plotly credentials are set as described above. 
# Make sure that you've run hidden code [1, 2, 3 and 4] before executing this cell. 
# Just uncomment the last line (remove the #) and hit <Ctrl + Enter>
# -----------------------------------------------------------------------------------------

#py.iplot(fig, filename='Figure4a_[pigExVivo]', config = config)


Click on me & hit [^] button (top toolbar) to display the code [5].


In [121]:
IFrame('//plot.ly/~agkar/146.embed', width=1000, height=800)


<i class="fa fa-sort-down" style="font-size: 1.5em;"></i>

In [98]:
# ----------------------------------------------------------------------------------------- NOTE
# You can regenerate this figure if your plotly credentials are set as described above. 
# Make sure that you've run hidden code [1, 2, 3, 4 and 5] before executing this cell. 
# Just uncomment the last line (remove the #) and hit <Ctrl + Enter>
# -----------------------------------------------------------------------------------------

preMolli_mtr  = getExcluded(MOLLI,'prefix','MTR', molliColor, molliFill, 'pre MOLLI')
preShmolli_mtr  = getExcluded(SHMOLLI,'prefix','MTR', shmolliColor, shmolliFill, 'pre SHMOLLI')
preSasha_mtr  = getExcluded(SASHA,'prefix','MTR', sashaColor, sashaFill, 'pre SASHA')

postMolli_mtr  = getExcluded(MOLLI,'postfix','MTR', molliColor, molliFill, 'post MOLLI')
postShmolli_mtr  = getExcluded(SHMOLLI,'postfix','MTR', shmolliColor, shmolliFill, 'post SHMOLLI')
postSasha_mtr  = getExcluded(SASHA,'postfix','MTR', sashaColor, sashaFill, 'post SASHA')

poolMolli_mtr  = getExcluded(MOLLI,'pool','MTR', molliColor, molliFill, 'pool MOLLI')
poolShmolli_mtr  = getExcluded(SHMOLLI,'pool','MTR', shmolliColor, shmolliFill, 'pool SHMOLLI')
poolSasha_mtr  = getExcluded(SASHA,'pool','MTR', sashaColor, sashaFill, 'pool SASHA')


fig = tools.make_subplots(rows=2, cols=3, shared_xaxes=False, shared_yaxes=True, vertical_spacing=0.15,horizontal_spacing= 0.01, print_grid=False)

traceAppend(preMolli_mtr,fig,1,1)
traceAppend(preShmolli_mtr,fig,1,1)
traceAppend(preSasha_mtr,fig,1,1)

traceAppend(postMolli_mtr,fig,1,2)
traceAppend(postShmolli_mtr,fig,1,2)
traceAppend(postSasha_mtr,fig,1,2)

traceAppend(poolMolli_mtr,fig,1,3)
traceAppend(poolShmolli_mtr,fig,1,3)
traceAppend(poolSasha_mtr,fig,1,3)

# Plotly follows raw-major order here. 
fig['layout']['xaxis1'].update(title='Prefixation MTR')
fig['layout']['xaxis5'].update(title='Postfixatoin MTR')
fig['layout']['xaxis6'].update(title='Pooled MTR')

fig['layout']['yaxis1'].update(title='T1 bias (ms)')

leg = dict(
        x= 0.1,
        y=-0.19,
        orientation="h",
        traceorder = "grouped",
        
        font=dict(
            family='sans-serif',
            size=8,
            color= '#FFFFFF'
        ),
        bgcolor='#173753',
        bordercolor='#FFFFFF',
        borderwidth=2
    )


fig['layout'].update(height=800, width=1000,title='Figure 4.2: Associations between MTR effects and T1 bias in explanted hearts.', legend= leg , hovermode = 'closest')

# Display robust correlation results START

fig['layout'].update(
    annotations=[
    # Sasha pre T2
    dict(
            x=2,
            y=4,
            xref= 'x4',
            yref= 'y2',
            text= preSasha_mtr[4], #+ '<br> <br>' + preMolli_t2[4]+ '<br> <br>' + preShmolli_t2[4],
            showarrow=False,
            font=dict(
                family='Lucida Console, monospace',
                size=11,
                color='#ffffff'
            ),
            align='left',
            bordercolor='#c7c7c7',
            borderwidth=0,
            borderpad=0,
            bgcolor= sashaColor,
            opacity=0.6
      
        ),
        # Molli pre T2
        dict(
            x=1.9,
            y=2,
            xref= 'x4',
            yref= 'y2',
            text= preMolli_mtr[4],
            showarrow=False,
            font=dict(
                family='Lucida Console, monospace',
                size=11,
                color='#ffffff'
            ),
            align='left',
            bordercolor='#c7c7c7',
            borderwidth=0,
            borderpad=0,
            bgcolor= molliColor,
            opacity=0.6
      
        ),
        #Shmolli pre T2
        dict(
            x=1.7,
            y=0,
            xref= 'x4',
            yref= 'y2',
            text= preShmolli_mtr[4],
            showarrow=False,
            font=dict(
                family='Lucida Console, monospace',
                size=11,
                color='#ffffff'
            ),
            align='left',
            bordercolor='#c7c7c7',
            borderwidth=0,
            borderpad=0,
            bgcolor= shmolliColor,
            opacity=0.6
      
        ),
                
        # Sasha post T2
    dict(
            x=1.86,
            y=4,
            xref= 'x5',
            yref= 'y2',
            text= postSasha_mtr[4], #+ '<br> <br>' + preMolli_t2[4]+ '<br> <br>' + preShmolli_t2[4],
            showarrow=False,
            font=dict(
                family='Lucida Console, monospace',
                size=11,
                color='#ffffff'
            ),
            align='left',
            bordercolor='#c7c7c7',
            borderwidth=0,
            borderpad=0,
            bgcolor= sashaColor,
            opacity=0.6
      
        ),
        # Molli post T2
        dict(
            x=2,
            y=2,
            xref= 'x5',
            yref= 'y2',
            text= postMolli_mtr[4],
            showarrow=False,
            font=dict(
                family='Lucida Console, monospace',
                size=11,
                color='#ffffff'
            ),
            align='left',
            bordercolor='#c7c7c7',
            borderwidth=0,
            borderpad=0,
            bgcolor= molliColor,
            opacity=0.6
      
        ),
        #Shmolli post T2
        dict(
            x=2,
            y=0,
            xref= 'x5',
            yref= 'y2',
            text= postShmolli_mtr[4],
            showarrow=False,
            font=dict(
                family='Lucida Console, monospace',
                size=11,
                color='#ffffff'
            ),
            align='left',
            bordercolor='#c7c7c7',
            borderwidth=0,
            borderpad=0,
            bgcolor= shmolliColor,
            opacity=0.6
      
        ),
                
        # Sasha pool T2
    dict(
            x=1.8,
            y=4,
            xref= 'x6',
            yref= 'y2',
            text= poolSasha_mtr[4], #+ '<br> <br>' + preMolli_t2[4]+ '<br> <br>' + preShmolli_t2[4],
            showarrow=False,
            font=dict(
                family='Lucida Console, monospace',
                size=11,
                color='#ffffff'
            ),
            align='left',
            bordercolor='#c7c7c7',
            borderwidth=0,
            borderpad=0,
            bgcolor= sashaColor,
            opacity=0.6
      
        ),
        # Molli pool T2
        dict(
            x=2,
            y=2,
            xref= 'x6',
            yref= 'y2',
            text= poolMolli_mtr[4],
            showarrow=False,
            font=dict(
                family='Lucida Console, monospace',
                size=11,
                color='#ffffff'
            ),
            align='left',
            bordercolor='#c7c7c7',
            borderwidth=0,
            borderpad=0,
            bgcolor= molliColor,
            opacity=0.6
      
        ),
        #Shmolli pool T2
        dict(
            x=2,
            y=0,
            xref= 'x6',
            yref= 'y2',
            text= poolShmolli_mtr[4],
            showarrow=False,
            font=dict(
                family='Lucida Console, monospace',
                size=11,
                color='#ffffff'
            ),
            align='left',
            bordercolor='#c7c7c7',
            borderwidth=0,
            borderpad=0,
            bgcolor= shmolliColor,
            opacity=0.6
      
        )
]
        )

# Display robust correlation results END 

fig['layout']['xaxis1'].update(
side='bottom',
position = -1)
fig['layout']['xaxis2'].update(
side='bottom',
position = -1)
fig['layout']['xaxis3'].update(
side='bottom',
position = -1)

fig['layout']['yaxis1'].update(
        domain = [0, 0.8] )
fig['layout']['yaxis2'].update(autorange=False,
        showgrid=False,
        zeroline=False,
        showline=False,
        autotick=True,
        ticks='',
        showticklabels=False,
        domain = [0.8, 1]
        )

fig['layout']['yaxis3'].update(
        domain = [0, 0.8] )

fig['layout']['yaxis4'].update(autorange=False,
        showgrid=False,
        zeroline=False,
        showline=False,
        autotick=True,
        ticks='',
        showticklabels=False,
        domain = [0.8, 1],
        range = [0, 10]  )

fig['layout']['yaxis5'].update(
        
        domain = [0, 0.8] )

fig['layout']['yaxis6'].update(autorange=False,
        showgrid=False,
        zeroline=False,
        showline=False,
        autotick=True,
        ticks='',
        showticklabels= False,
        domain = [0.8, 1],
       range = [0, 10] )

fig['layout']['xaxis4'].update(autorange=False,
        showgrid=False,
        zeroline=False,
        showline=False,
        autotick=True,
        ticks='',
        showticklabels=False                   
        )

fig['layout']['xaxis5'].update(autorange=False,
        showgrid=False,
        zeroline=False,
        showline=False,
        autotick=True,
        ticks='',
        showticklabels=False
        )
    
fig['layout']['xaxis6'].update(autorange=False,
        showgrid=False,
        zeroline=False,
        showline=False,
        autotick=True,
        ticks='',
        showticklabels=False                       
        )

config={'showLink': False}

print('Click on me & hit [^] button (top toolbar) to display the code [6].')

# ----------------------------------------------------------------------------------------- NOTE
# You can regenerate this figure if your plotly credentials are set as described above. 
# Make sure that you've run hidden code [1, 2, 3, 4 and 5] before executing this cell. 
# Just uncomment the last line (remove the #) and hit <Ctrl + Enter>
# -----------------------------------------------------------------------------------------

#py.iplot(fig, filename='Figure4b_pigExVivo', config = config)


Click on me & hit [^] button (top toolbar) to display the code [6].


<img src="copyright_neuropoly.jpg">